In [1]:
import pandas as pd
import numpy as np
import codecs
import sys
import nltk
from bs4 import BeautifulSoup
import re

In [2]:
#pip install nltk
#pip install spacy
#python -m spacy download de_core_news_sm

In [3]:
#%config IPCompleter.greedy=True

doc = codecs.open('data/coronaContent.csv','rU','UTF-16') #open for reading with "universal" type set

df = pd.read_csv(doc, sep='\t')
print(df.shape)
df.head()

(1430, 10)


,ID_GodotObject,documentXML,url,author,RessortName,source,sourceCategory,ObjectType,publishingDate,titletext
0,2000120059197,"﻿<document id=""2000120059197"" homeRessortId=""2...",/jetzt/livebericht/2000120059197/anschober-erw...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-09-17 06:54:54.87,Verschärfte Corona-Maßnahmen: Private Treffen ...
1,2000122402488,"﻿<document id=""2000122402488"" homeRessortId=""2...",/jetzt/livebericht/2000122402488/massentests-i...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-12-11 06:08:34.23,"Maskenpflicht im Freien und am Arbeitsplatz, S..."
2,2000120089418,"﻿<document id=""2000120089418"" homeRessortId=""2...",/jetzt/livebericht/2000120089418/mehr-als-30-m...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-09-18 06:31:42.12,"Wien richtet neue Teststraße ein, österreichwe..."
3,2000122440975,"﻿<document id=""2000122440975"" homeRessortId=""2...",/jetzt/livebericht/2000122440975/maskenpflicht...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-12-12 07:21:48.45,Geringe Testbeteiligung in Wien zeichnet sich ...
4,2000122442386,"﻿<document id=""2000122442386"" homeRessortId=""2...",/2000122442386/Treffen-mit-sechs-Personen-zu-S...,(keine Angabe),Coronavirus,AutorIn,Online only,Artikel,2020-12-12 07:13:09.00,Wie viele Menschen kann man nun zu Weihnachten...


In [4]:
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna()
df = df.applymap(str)
df.ID_GodotObject = df.ID_GodotObject.astype('int64')
df['publishingDate'] = pd.to_datetime(df['publishingDate'])
df = df.sort_values(by='publishingDate').reset_index(drop=True)


#df = df.dropna()

print(df.shape)
print(df.dtypes)
df.head()


(103, 10)
ID_GodotObject             int64
documentXML               object
url                       object
author                    object
RessortName               object
source                    object
sourceCategory            object
ObjectType                object
publishingDate    datetime64[ns]
titletext                 object
dtype: object


,ID_GodotObject,documentXML,url,author,RessortName,source,sourceCategory,ObjectType,publishingDate,titletext
0,2000115059032,"﻿<document id=""2000115059032"" homeRessortId=""2...",/2000115059032/Maskenpflicht-fuer-medizinische...,(keine Angabe),Coronavirus,Redaktion,Online only,Artikel,2020-02-26 16:50:27.000,Maskenpflicht für medizinisches Personal im Um...
1,2000116305030,"﻿<document id=""2000116305030"" homeRessortId=""2...",/jetzt/livebericht/2000116305030/regierung-zie...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-03-30 05:36:31.240,Schrittweise Einführung von Maskenpflicht in d...
2,2000116325081,"﻿<document id=""2000116325081"" homeRessortId=""2...",/2000116325081/Regierung-stellt-weitere-Corona...,(keine Angabe),Coronavirus,Redaktion,Online only,Artikel,2020-03-30 13:17:32.120,"Regierung setzt auf Masken beim Einkauf, Tests..."
3,2000116346340,"﻿<document id=""2000116346340"" homeRessortId=""2...",/jetzt/livebericht/2000116346340/schrittweise-...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-03-31 05:34:39.260,Maskenpflicht im Supermarkt spätestens ab Mont...
4,2000116371728,"﻿<document id=""2000116371728"" homeRessortId=""2...",/jetzt/livebericht/2000116371728/mittwoch-um-1...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveChat,2020-03-31 16:15:23.100,"Sonderbeauftragter Clemens Auer: ""Italienische..."


To preprocess a HTML news article file in Python for building an opinion mining model on the topic of mask restrictions to prevent the spread of coronavirus, you would first need to use the BeautifulSoup library to parse the HTML file and extract the text content from the article. This would allow you to access the article's text as a string in Python.

In [5]:
text = []

for row in df.itertuples():
    soup = BeautifulSoup(row.documentXML, 'html.parser')
    text.append(soup.get_text())
    #print(soup.get_text())

df['content'] = pd.Series(text, copy=False)
df = df.drop('documentXML', axis=1)
df.head()

,ID_GodotObject,url,author,RessortName,source,sourceCategory,ObjectType,publishingDate,titletext,content
0,2000115059032,/2000115059032/Maskenpflicht-fuer-medizinische...,(keine Angabe),Coronavirus,Redaktion,Online only,Artikel,2020-02-26 16:50:27.000,Maskenpflicht für medizinisches Personal im Um...,﻿2.0Sars-CoV-2Maskenpflicht für medizinisches ...
1,2000116305030,/jetzt/livebericht/2000116305030/regierung-zie...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-03-30 05:36:31.240,Schrittweise Einführung von Maskenpflicht in d...,﻿2.0CoronavirusSchrittweise Einführung von Mas...
2,2000116325081,/2000116325081/Regierung-stellt-weitere-Corona...,(keine Angabe),Coronavirus,Redaktion,Online only,Artikel,2020-03-30 13:17:32.120,"Regierung setzt auf Masken beim Einkauf, Tests...",﻿2.0Kein Ende in SichtRegierung setzt auf Mask...
3,2000116346340,/jetzt/livebericht/2000116346340/schrittweise-...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveBericht,2020-03-31 05:34:39.260,Maskenpflicht im Supermarkt spätestens ab Mont...,﻿2.0CoronavirusMaskenpflicht im Supermarkt spä...
4,2000116371728,/jetzt/livebericht/2000116371728/mittwoch-um-1...,(keine Angabe),Coronavirus,Redaktion,Online only,LiveChat,2020-03-31 16:15:23.100,"Sonderbeauftragter Clemens Auer: ""Italienische...",﻿2.0CoronavirusSonderbeauftragter Clemens Auer...


Next, you would need to clean and preprocess the text data by removing punctuation, stopwords, and other irrelevant information. You could use the NLTK library's built-in functions for this purpose, such as the word_tokenize() and stopwords.words() functions.

- Import the necessary NLTK modules, including word_tokenize() and stopwords.words().
- Define the German news article that you want to process.
- Use stopwords.words() to get a list of German stopwords.
- Use word_tokenize() to tokenize the article into individual words.
- Remove the stopwords from the list of tokens.
- Identify words and phrases in the remaining list of tokens that express opinions or viewpoints.

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk import word_tokenize

text = df['content'][2]
regex = r"<[^>]*>"
# Delete all of the HTML tags
text = re.sub(regex, " ", text)

words = nltk.word_tokenize(text)
new_words= [word for word in words if word.isalnum()]
#print(new_words)

# Get a list of German stopwords
stop_words = stopwords.words("german")

# Remove the stopwords from the list of tokens
filtered_words = [new_word for new_word in new_words if new_word not in stop_words]

#Split two concatinated Nouns -> SichtRegierung into Sicht and Regierung
filtered_words1 = []
pattern = r'[A-Z][a-z]+[A-Z][a-z]+'
pattern2 = r'([A-Z][a-z]+)([A-Z][a-z]+)'
for w in filtered_words:
    match = re.search(pattern, w)
    if match:
        words = re.split(pattern2, w)
        words = [i for i in words if i]
        filtered_words1 += words
    else:
        filtered_words1.append(w)



filtered_words = filtered_words1
print(filtered_words)


['Ende', 'Sicht', 'Regierung', 'setzt', 'Masken', 'beim', 'Einkauf', 'Tests', 'Freistellungen', 'Montag', 'wurden', 'neue', 'Maßnahmen', 'Eindämmung', 'Coronavirus', 'vorgestellt', 'Wir', 'Ziel', 'weit', 'entfernt', 'sagt', 'Gesundheitsminister', 'Ausschnitte', 'Pressekonferenz', 'Video', 'Die', 'Bundesregierung', 'Montag', 'Rahmen', 'Pressekonferenz', 'weitere', 'Maßnahmen', 'Eindämmung', 'Coronavirus', 'verkündet', 'Es', 'gelte', 'Überforderung', 'Gesundheitssystems', 'Mitteln', 'verhindern', 'sagte', 'Bundeskanzler', 'Sebastian', 'Kurz', 'ÖVP', 'Dies', 'solle', 'verstärkten', 'Schutz', 'besonders', 'gefährdeter', 'Gruppen', 'konsequente', 'Durchsetzung', 'Maßnahmen', 'Exekutive', 'verpflichtende', 'Tragen', 'Masken', 'erreicht', 'Wir', 'Ziel', 'weit', 'entfernt', 'sagte', 'Gesundheitsminister', 'Rudolf', 'Anschober', 'Grüne', 'Ein', 'Ende', 'Maßnahmen', 'sei', 'demnach', 'länger', 'Sicht', 'Um', 'Schutz', 'älteren', 'Menschen', 'Menschen', 'Vorerkrankungen', 'verbessern', 'sollen', 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


After preprocessing the text data, you could use techniques such as tokenization and stemming to convert the text into a format that can be easily analyzed by the model. The NLTK library provides functions such as PorterStemmer() and LancasterStemmer() for this purpose.



There are several stemmers available in NLTK for German, each with their own strengths and weaknesses. The best stemmer to use for a given task will depend on the specific goals and requirements of your project. For example, if you are working with written German news articles targeting the topic of Corona, you might want to use the Snowball stemmer, which is specifically designed to provide good stemming results for German. Other popular stemmers for German include the Ludwig stemmer and the German stemmer.

It's important to note that stemmers are not always necessary or even desirable for natural language processing tasks. In many cases, it may be better to use lemmatization instead of stemming, as lemmatization is more accurate and produces more meaningful word forms. Lemmatization is the process of determining the base form of a word, called the lemma, which can be useful for tasks like text classification and information extraction.

Overall, the best stemmer to use for a given task will depend on the specific goals and requirements of your project, as well as the nature of the text you are working with. It's always a good idea to experiment with different stemmers and evaluate their performance on your specific dataset to determine which one works best for your needs.

In [7]:
#from nltk.stem import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer
#from nltk.tokenize import sent_tokenize, word_tokenize

# Create a SnowballStemmer instance for the German language
#stemmer = SnowballStemmer("german")

# Apply stemming to the list of tokens
#stemmed_tokens = [stemmer.stem(token) for token in filtered_words]

#print(stemmed_tokens)

To use lemmatization in NLTK for a German news article targeting the topic of Corona, you can use the WordNetLemmatizer class from NLTK. This class uses the WordNet database to perform lemmatization, and it is compatible with the Penn Treebank part-of-speech tags, which are commonly used in natural language processing tasks.

Here is an example of how you might use the WordNetLemmatizer class to perform lemmatization on a German news article:

In [8]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
# Initialize the lemmatizer
#lemmatizer = WordNetLemmatizer()
# Part-of-speech tag the tokens
#tagged_tokens = nltk.pos_tag(filtered_words)
# Lemmatize the tagged tokens
import spacy
#python3 -m spacy download fr_core_news_md
nlp = spacy.load('de_core_news_md')
filtered_lemma = []
for w in filtered_words:
     result = ' '.join([token.lemma_ for token in nlp(w)]) 
     filtered_lemma.append(result)
print(filtered_lemma)
print(len(filtered_lemma))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['Ende', 'Sicht', 'Regierung', 'setzen', 'Maske', 'bei', 'Einkauf', 'Test', 'Freistellung', 'Montag', 'werden', 'neu', 'Maßnahme', 'Eindämmung', 'Coronavirus', 'vorstellen', 'wir', 'Ziel', 'weit', 'entfernt', 'sagen', 'Gesundheitsminister', 'Ausschnitte', 'Pressekonferenz', 'Video', 'der', 'Bundesregierung', 'Montag', 'Rahmen', 'Pressekonferenz', 'weit', 'Maßnahme', 'Eindämmung', 'Coronavirus', 'verkünden', 'es', 'gelten', 'Überforderung', 'Gesundheitssystem', 'Mittel', 'verhindern', 'sagen', 'Bundeskanzler', 'Sebastian', 'kurz', 'ÖVP', 'dieser', 'sollen', 'verstärken', 'Schutz', 'besonders', 'gefährdet', 'Gruppe', 'konsequent', 'Durchsetzung', 'Maßnahme', 'Exekutive', 'verpflichtenden', 'tragen', 'Maske', 'erreichen', 'wir', 'Ziel', 'weit', 'entfernt', 'sagen', 'Gesundheitsminister', 'Rudolf', 'Anschober', 'grüne', 'ein', 'Ende', 'Maßnahme', 'sein', 'demnach', 'lang', 'Sicht', 'um', 'Schutz', 'alt', 'Mensch', 'Mensch', 'Vorerkrankung', 'verbessern', 'sollen', 'Bevölkerungsgruppe', 've

In [9]:
print(stop_words)

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein', 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euc

do that on the whole frame

In [10]:
df1 = df[['ID_GodotObject','content','titletext','author','source','publishingDate']]

def preprocessing(text):

        regex = r"<[^>]*>"
        # Delete all of the HTML tags
        text = re.sub(regex, " ", text)

        words = nltk.word_tokenize(text)
        new_words= [word for word in words if word.isalnum()]
        stop_words = stopwords.words("german")
        filtered_words = [new_word for new_word in new_words if new_word not in stop_words]
        
        #remove concatenate Nouns
        filtered_words1 = []
        pattern = r'[A-Z][a-z]+[A-Z][a-z]+'
        pattern2 = r'([A-Z][a-z]+)([A-Z][a-z]+)'
        for w in filtered_words:
            match = re.search(pattern, w)
            if match:
                words = re.split(pattern2, w)
                words = [i for i in words if i]
                filtered_words1 += words
            else:
                filtered_words1.append(w)
        filtered_words = filtered_words1
    
        nlp = spacy.load('de_core_news_md')
        filtered_lemma = []
        for w in filtered_words:
            result = ' '.join([token.lemma_ for token in nlp(w)]) 
            filtered_lemma.append(result)
        
        return ' '.join(filtered_lemma)

    

#df1[['content']] = df1[['content']].apply(lambda x: preprocessing(str(x)))
#df1[['titletext']] = df1[['titletext']].apply(lambda x: preprocessing(str(x)))
df1['p_content'] = df1['content'].apply(lambda x: preprocessing(str(x)))
df1['p_titletext'] = df1['titletext'].apply(lambda x: preprocessing(str(x)))
df1.head()

#df1.isnull().sum()

/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_59202/1080105394.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['p_content'] = df1['content'].apply(lambda x: preprocessing(str(x)))
/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_59202/1080105394.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['p_titletext'] = df1['titletext'].apply(lambda x: preprocessing(str(x)))


,ID_GodotObject,content,titletext,author,source,publishingDate,p_content,p_titletext
0,2000115059032,﻿2.0Sars-CoV-2Maskenpflicht für medizinisches ...,Maskenpflicht für medizinisches Personal im Um...,(keine Angabe),Redaktion,2020-02-26 16:50:27.000,medizinisch Personal Umgang Labor müssen Probe...,Maskenpflicht medizinisch Personal Umgang Coro...
1,2000116305030,﻿2.0CoronavirusSchrittweise Einführung von Mas...,Schrittweise Einführung von Maskenpflicht in d...,(keine Angabe),Redaktion,2020-03-30 05:36:31.240,Einführung Maskenpflicht Regierung verschärfen...,schrittweise Einführung Maskenpflicht Öffentli...
2,2000116325081,﻿2.0Kein Ende in SichtRegierung setzt auf Mask...,"Regierung setzt auf Masken beim Einkauf, Tests...",(keine Angabe),Redaktion,2020-03-30 13:17:32.120,Ende Sicht Regierung setzen Maske bei Einkauf ...,Regierung setzen Maske bei Einkauf Test Freist...
3,2000116346340,﻿2.0CoronavirusMaskenpflicht im Supermarkt spä...,Maskenpflicht im Supermarkt spätestens ab Mont...,(keine Angabe),Redaktion,2020-03-31 05:34:39.260,Supermarkt spätestens ab Montag Entscheidung M...,Maskenpflicht Supermarkt spätestens ab Montag ...
4,2000116371728,﻿2.0CoronavirusSonderbeauftragter Clemens Auer...,"Sonderbeauftragter Clemens Auer: ""Italienische...",(keine Angabe),Redaktion,2020-03-31 16:15:23.100,Clemens Auer italienisch spanisch Verhältnis v...,Sonderbeauftragter Clemens Auer italienisch sp...


In [11]:

df1 = df1.drop(['content','titletext'],axis=1)
df1.rename(columns = {"p_content":"content","p_titletext":"titletext"}, inplace = True)


In [12]:
df1.to_csv('data/p_content.csv', encoding='utf-8', index=False)